# Federated IDS (4 clients) with TenSEAL (CKKS) encrypted weight aggregation



In [ ]:
!pip install TenSEAL

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 43.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt

try:
    import tenseal as ts
except Exception as e:
    print('tenseal not available. Install tenseal (pip install tenseal) to use encrypted aggregation.')
    ts = None


tenseal not available. Install tenseal (pip install tenseal) to use encrypted aggregation.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
SAMPLING_FRACTION = 0.2

DATA_DIR = '/content/drive/MyDrive/datasets/CICIDS2017'
csv_files = glob.glob(os.path.join(DATA_DIR, '*.csv'))
if not csv_files:
    raise FileNotFoundError(f'No CSV files found in {DATA_DIR}. Please upload them and re-run.')

print(f'Found {len(csv_files)} CSV files. Reading and concatenating...')
dfs = [pd.read_csv(file, low_memory=False) for file in csv_files]
df = pd.concat(dfs, ignore_index=True)
print('Raw combined shape:', df.shape)


if SAMPLING_FRACTION < 1.0:
    df_sampled = df.sample(frac=SAMPLING_FRACTION, random_state=42)
    print(f'Dataset sampled to {SAMPLING_FRACTION*100:.1f}% of original size.')
    print('Sampled shape:', df_sampled.shape)
else:
    df_sampled = df
    print('No sampling applied (SAMPLING_FRACTION is 1.0).')

df = df_sampled


if 'Label' in df.columns:
    label_col = 'Label'
elif ' Label' in df.columns:
    label_col = ' Label'
else:
    label_col = df.columns[-1]
    print('Using last column as label:', label_col)

non_numeric = df.select_dtypes(exclude=[np.number]).columns.tolist()
non_numeric = [c for c in non_numeric if c != label_col]
df = df.drop(columns=non_numeric, errors='ignore')

unique_labels = df[label_col].unique()
label_mapping = {}
current_index = 0
for lbl in unique_labels:
    lbl_str = str(lbl).strip().upper()
    if lbl_str.startswith("BENIGN"):
        label_mapping[lbl] = 0
    else:
        current_index += 1
        label_mapping[lbl] = current_index

df[label_col] = df[label_col].map(label_mapping)
df = df.dropna(subset=[label_col])

# Feature / label split
X = df.drop(columns=[label_col]).values
y = df[label_col].values

# Replace inf/nan with column means
X = np.where(np.isinf(X), np.nan, X)
col_means = np.nanmean(X, axis=0)
inds = np.where(np.isnan(X))
X[inds] = np.take(col_means, inds[1])

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)
num_classes = len(np.unique(y))


X_fl_train, X_test_full, y_fl_train, y_test_full = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print('--- Data Summary ---')
print(f'Total features shape after sampling/cleaning: {X.shape}')
print(f'FL Training Data Shape: {X_fl_train.shape}')
print(f'Global Test Data Shape: {X_test_full.shape}')
print(f'Number of Classes: {num_classes}')
print("Fine-grained label distribution (FL Training Set):")
print(pd.Series(y_fl_train).value_counts())
print('✅ Preprocessing and Global Split complete.')

Found 9 CSV files. Reading and concatenating...
Raw combined shape: (5349020, 79)
Dataset sampled to 20.0% of original size.
Sampled shape: (1069804, 79)
--- Data Summary ---
Total features shape after sampling/cleaning: (1069804, 78)
FL Training Data Shape: (855843, 78)
Global Test Data Shape: (213961, 78)
Number of Classes: 14
Fine-grained label distribution (FL Training Set):
0     679945
2      74108
3      50664
4      40993
5       2521
1       1903
6       1896
9       1746
10       720
8        646
7        491
11       194
12        11
13         5
Name: count, dtype: int64
✅ Preprocessing and Global Split complete.


In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

In [ ]:
!pip install -U imbalanced-learn


In [ ]:
import numpy as np
from collections import Counter
from imblearn.over_sampling import SMOTE

print("Original label distribution (FL Training Set):", Counter(y_fl_train))


TARGET_COUNT = 25000
current_counts = Counter(y_fl_train)
sampling_strategy = {}

for label, count in current_counts.items():

    if label != 0:
        sampling_strategy[label] = max(count, TARGET_COUNT)
    else:

        sampling_strategy[label] = count


smote = SMOTE(random_state=42, sampling_strategy=sampling_strategy, k_neighbors=2)
X_res, y_res = smote.fit_resample(X_fl_train, y_fl_train)

print("After CONTROLLED SMOTE on FL Training Data:")
print(Counter(y_res))
print(f"Total samples for FL training after SMOTE: {len(y_res)}")

# Non-IID Split Function (Dirichlet)

def split_noniid(X, y, num_clients=4, alpha=0.5):
    np.random.seed(42)
    labels = np.unique(y)
    idx_per_label = {label: np.where(y == label)[0].tolist() for label in labels}
    clients_idx = {i: [] for i in range(num_clients)}

    for label, idxs in idx_per_label.items():
        np.random.shuffle(idxs)
        proportions = np.random.dirichlet([alpha]*num_clients)
        counts = (proportions * len(idxs)).astype(int)
        while counts.sum() < len(idxs):
            counts[np.argmax(proportions)] += 1
        ptr = 0
        for c in range(num_clients):
            cnt = counts[c]
            if cnt > 0:
                clients_idx[c].extend(idxs[ptr:ptr+cnt])
                ptr += cnt

    clients = []
    for c in range(num_clients):
        idcs = clients_idx[c]
        clients.append((X[idcs], y[idcs]))
    return clients

num_clients = 4
clients = split_noniid(X_res, y_res, num_clients=num_clients, alpha=0.5)

print("\n--- Client Data Distribution ---")
for i, (Xc, yc) in enumerate(clients):
    print(f"Client {i+1}: {len(yc)} samples | label distribution: {Counter(yc)}")
print('✅ Client partitioning complete.')

Original label distribution (FL Training Set): Counter({np.int64(0): 679945, np.int64(2): 74108, np.int64(3): 50664, np.int64(4): 40993, np.int64(5): 2521, np.int64(1): 1903, np.int64(6): 1896, np.int64(9): 1746, np.int64(10): 720, np.int64(8): 646, np.int64(7): 491, np.int64(11): 194, np.int64(12): 11, np.int64(13): 5})
After CONTROLLED SMOTE on FL Training Data:
Counter({np.int64(0): 679945, np.int64(2): 74108, np.int64(3): 50664, np.int64(4): 40993, np.int64(5): 25000, np.int64(10): 25000, np.int64(6): 25000, np.int64(1): 25000, np.int64(8): 25000, np.int64(7): 25000, np.int64(9): 25000, np.int64(11): 25000, np.int64(13): 25000, np.int64(12): 25000})
Total samples for FL training after SMOTE: 1095710

--- Client Data Distribution ---
Client 1: 429055 samples | label distribution: Counter({np.int64(0): 269260, np.int64(3): 41803, np.int64(4): 25235, np.int64(6): 22128, np.int64(13): 18273, np.int64(11): 14639, np.int64(10): 11372, np.int64(1): 11087, np.int64(5): 4985, np.int64(2): 4

In [ ]:

class ImprovedMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, output_dim=15):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x):
        return self.net(x)

def get_model(input_dim, num_classes):
    """Returns an initialized ImprovedMLP model with the correct dimensions."""

    return ImprovedMLP(input_dim, output_dim=num_classes)

In [ ]:
def model_to_vector(state_dict):
    vec = []
    shapes = {}
    for k, v in state_dict.items():
        arr = v.cpu().numpy().ravel()
        shapes[k] = v.shape
        vec.append(arr)
    flat = np.concatenate(vec).astype(np.float64)
    return flat, shapes

def vector_to_model(state_dict_template, flat_vec, shapes):
    new_state = {}
    ptr = 0
    for k in state_dict_template.keys():
        num = int(np.prod(shapes[k]))
        slice_ = flat_vec[ptr:ptr+num]
        new_state[k] = torch.tensor(slice_.reshape(shapes[k]), dtype=state_dict_template[k].dtype)
        ptr += num
    return new_state

def create_tenseal_context():
    if ts is None:
        raise RuntimeError('tenseal not installed')
    ctx = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 60])
    ctx.generate_galois_keys()
    ctx.global_scale = 2**40
    return ctx

def encrypt_vector(ctx, vec):
    return ts.ckks_vector(ctx, vec)

def decrypt_vector(enc_vec):
    return np.array(enc_vec.decrypt())


In [ ]:
def train_local(model, X_train, y_train, epochs=3, batch_size=64, lr=1e-3, device='cpu'):
    model = model.to(device)
    model.train()
    opt = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    loader = DataLoader(ds, batch_size=batch_size, shuffle=True)
    for e in range(epochs):
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            opt.step()
    return model.state_dict()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import tenseal as ts
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def federated_train_with_encryption(clients, input_dim, num_classes,
                                    rounds=7, local_epochs=5, batch_size=32, device='cpu'):
    """
    Federated Learning with encryption, mini-batching, weighted loss, and improved stability.
    """
    print(f"Starting Federated Learning with Encryption — Rounds: {rounds}, Clients: {len(clients)}")

-
    global_model = get_model(input_dim, num_classes).to(device)


    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=16384,
        coeff_mod_bit_sizes=[60, 40, 40, 40, 40, 40, 60]
    )
    context.global_scale = 2**40
    context.generate_galois_keys()

    for r in range(rounds):
        print(f"\nRound {r+1}/{rounds} — clients: {len(clients)}")
        encrypted_updates = []
        global_weights_dict = global_model.state_dict()


        global_weights_vector = torch.cat([p.data.view(-1) for p in global_model.parameters()]).cpu().numpy()

        for cid, (Xc, yc) in enumerate(clients):
            print(f" Client {cid}: training on {len(Xc)} samples")

            local_model = get_model(input_dim, num_classes).to(device)
            local_model.load_state_dict(global_weights_dict)
            optimizer = optim.Adam(local_model.parameters(), lr=0.0001)

            class_counts = np.bincount(yc, minlength=num_classes)

            weights = 1.0 / (class_counts + 1e-6)
            weights = torch.tensor(weights, dtype=torch.float32).to(device)
            criterion = nn.CrossEntropyLoss(weight=weights)

            dataset = TensorDataset(torch.tensor(Xc, dtype=torch.float32),
                                    torch.tensor(yc, dtype=torch.long))
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

            local_model.train()
            for epoch in range(local_epochs):
                for Xbatch, ybatch in loader:
                    Xbatch, ybatch = Xbatch.to(device), ybatch.to(device)
                    optimizer.zero_grad()
                    out = local_model(Xbatch)
                    loss = criterion(out, ybatch)
                    loss.backward()
                    optimizer.step()

            local_weights_vector = torch.cat([p.data.view(-1) for p in local_model.parameters()]).cpu().numpy()
            model_update = local_weights_vector - global_weights_vector

            enc_update_vector = ts.ckks_vector(context, model_update.tolist())
            encrypted_updates.append(enc_update_vector)


        enc_sum_updates = encrypted_updates[0]
        for enc in encrypted_updates[1:]:
            # Homomorphic addition
            enc_sum_updates += enc

        enc_avg_update = enc_sum_updates * (1.0 / len(clients))
        decrypted_avg_update = np.array(enc_avg_update.decrypt())
        print(" Decrypted aggregated UPDATE sample (first 10 elements):", np.round(decrypted_avg_update[:10], 6))

        new_global_weights = global_weights_vector + decrypted_avg_update
        idx = 0
        new_state = {}
        for name, param in global_model.state_dict().items():
            size = param.numel()
            new_state[name] = torch.tensor(new_global_weights[idx:idx+size], dtype=param.dtype).view(param.shape)
            idx += size
        global_model.load_state_dict(new_state)
        print(f" Round {r+1} global model updated.")

    return global_model

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch
from torch.utils.data import TensorDataset, DataLoader


if 'clients' in globals() and len(clients) >= 1:

    input_dim = clients[0][0].shape[1]

    try:
        global_model = federated_train_with_encryption(
            clients, input_dim, num_classes=num_classes, rounds=7, local_epochs=5, device='cpu'
        )
    except RuntimeError as e:
        print('RuntimeError:', e)
        print('If TenSEAL is not installed, you can still run plain aggregation by removing TenSEAL calls.')

        global_model = get_model(input_dim, num_classes).to('cpu')

    global_model.eval()

    test_dataset = TensorDataset(torch.tensor(X_test_full, dtype=torch.float32),
                                 torch.tensor(y_test_full, dtype=torch.long))
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    all_preds = []
    all_labels = []
    with torch.no_grad():
        for xb, yb in test_loader:
            out = global_model(xb)
            preds = torch.argmax(out, dim=1)
            all_preds.append(preds.cpu())
            all_labels.append(yb.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()


    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    prec = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    rec = recall_score(all_labels, all_preds, average='weighted', zero_division=0)

    print('\n=== Final Global Model Metrics on UNSEEN Test Set ===')
    print(f' Accuracy:  {acc:.4f}')
    print(f' F1-score:  {f1:.4f}')
    print(f' Precision: {prec:.4f}')
    print(f' Recall:    {rec:.4f}')

    print('\n=== Per-client evaluation (on local training data) ===')
    for cid, (Xc, yc) in enumerate(clients):
        client_dataset = TensorDataset(torch.tensor(Xc, dtype=torch.float32),
                                       torch.tensor(yc, dtype=torch.long))
        client_loader = DataLoader(client_dataset, batch_size=64, shuffle=False)

        client_preds = []
        client_labels = []
        with torch.no_grad():
            for xb, yb in client_loader:
                out = global_model(xb)
                preds_c = torch.argmax(out, dim=1)
                client_preds.append(preds_c.cpu())
                client_labels.append(yb.cpu())

        client_preds = torch.cat(client_preds).numpy()
        client_labels = torch.cat(client_labels).numpy()

        acc_c = accuracy_score(client_labels, client_preds)
        f1_c = f1_score(client_labels, client_preds, average='weighted', zero_division=0)
        prec_c = precision_score(client_labels, client_preds, average='weighted', zero_division=0)
        rec_c = recall_score(client_labels, client_preds, average='weighted', zero_division=0)
        print(f' Client {cid} -> Accuracy: {acc_c:.4f}, F1: {f1_c:.4f}, Precision: {prec_c:.4f}, Recall: {rec_c:.4f}')
else:
    print('Clients not prepared. Ensure dataset was loaded and clients created.')

Starting Federated Learning with Encryption — Rounds: 7, Clients: 4

Round 1/7 — clients: 4
 Client 0: training on 429055 samples
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
 Client 1: training on 96009 samples
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
 Client 2: training on 214230 samples
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
 Client 3: training on 356416 samples
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those ope